In [1]:
import pretty_midi
import pandas as pd
import numpy as np
import matplotlib
import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.models import load_model


Using TensorFlow backend.


In [7]:
midi=pretty_midi.PrettyMIDI('001.mid',220)
df=pd.DataFrame(midi.get_piano_roll(fs=4))
#print(df)
#df.to_csv('out11.csv')

#data=df.values
#print(df[1:10])

#print('df')
target1 = df.transpose()


target1[target1>0]=1
target1.to_csv('out22.csv')
#print(target1)

target=target1[1:]
data=target1[:-1]


target11=target.values
data=data.values

print(target.shape)

data = np.reshape(data,(63,1,128))
target = np.reshape(target11,(63,1,128))
print(data)

(63, 128)
[[[ 0.  0.  0. ...,  0.  0.  0.]]

 [[ 0.  0.  0. ...,  0.  0.  0.]]

 [[ 0.  0.  0. ...,  0.  0.  0.]]

 ..., 
 [[ 0.  0.  0. ...,  0.  0.  0.]]

 [[ 0.  0.  0. ...,  0.  0.  0.]]

 [[ 0.  0.  0. ...,  0.  0.  0.]]]


In [8]:
#from sklearn.model_selection import train_test_split

In [9]:
#X_train, X_test, y_train, y_test = train_test_split(data, target,
                                                    test_size=0.2)
#data.shape

IndentationError: unexpected indent (<ipython-input-9-3e506b844443>, line 2)

In [ ]:
#X_train.shape
#X_test.shape

In [ ]:
model = Sequential()
model.add(LSTM(128,input_shape=(1,128),return_sequences=True))
model.add(LSTM(128,return_sequences=True))
model.add((Dense(128)))
model.compile(loss='mean_squared_error',optimizer='adam',metrics=['acc'])
model.fit(data,target,epochs=20,batch_size=1,verbose=2)

In [ ]:
#prediction
yhat = model.predict(data)
print(yhat.shape,'yhat11111')
yhat1 = np.reshape(yhat,(63,128))
print(yhat1.shape,'yhat11111')
yhat1[yhat1>=0.5]=1
yhat1[yhat1<0.5]=0
df=pd.DataFrame(yhat1)
df.to_csv('out11.csv')


#i=(0.5 < yhat1)
#yhat1=yhat1[yhat1<0.1]

#print(yhat)
#print(model.get_weights())


yhat2 = np.transpose(yhat1)



#print(df)
#midi.fluidsynth(fs=44100)
#df=pd.DataFrame(midi.get_piano_roll(1))
#print(df.describe())
#df=pd.DataFrame(midi.get_piano_roll(384),index=midi.get_piano_roll(384))
#print(midi.get_end_time())
#time=int(midi.get_end_time())


def piano_roll_to_pretty_midi(piano_roll, fs=4, program=0):
    '''Convert a Piano Roll array into a PrettyMidi object
     with a single instrument.
    Parameters
    ----------
    piano_roll : np.ndarray, shape=(128,frames), dtype=int
        Piano roll of one instrument
    fs : int
        Sampling frequency of the columns, i.e. each column is spaced apart
        by ``1./fs`` seconds.
    program : int
        The program number of the instrument.
    Returns
    -------
    midi_object : pretty_midi.PrettyMIDI
        A pretty_midi.PrettyMIDI class instance describing
        the piano roll.
    '''
    notes, frames = piano_roll.shape
    pm = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=program)

    # pad 1 column of zeros so we can acknowledge inital and ending events
    piano_roll = np.pad(piano_roll, [(0, 0), (1, 1)], 'constant')

    # use changes in velocities to find note on / note off events
    velocity_changes = np.nonzero(np.diff(piano_roll).T)

    # keep track on velocities and note on times
    prev_velocities = np.zeros(notes, dtype=int)
    note_on_time = np.zeros(notes)

    for time, note in zip(*velocity_changes):
        # use time + 1 because of padding above
        velocity = piano_roll[note, time + 1]
        time = time / fs
        if velocity > 0:
            if prev_velocities[note] == 0:
                note_on_time[note] = time
                prev_velocities[note] = velocity
        else:
            pm_note = pretty_midi.Note(
                velocity=prev_velocities[note],
                pitch=note,
                start=note_on_time[note],
                end=time)
            instrument.notes.append(pm_note)
            prev_velocities[note] = 0
    pm.instruments.append(instrument)
    return pm


midi=piano_roll_to_pretty_midi(yhat2)


midi.write('fiiinnnaal2.mid')